# 基本評価指標の実装

このノートブックでは、分類問題の基本評価指標（Accuracy、Precision、Recall、Specificity）を実装し、理解を深めます。

## 学習目標
- 基本評価指標の理論的理解
- 手動実装とscikit-learnの比較
- 実データでの指標計算
- 指標の解釈と活用


In [ ]:
# 必要なライブラリのインポート
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_classification, load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                           f1_score, confusion_matrix, classification_report)
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# 日本語フォントの設定
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_style("whitegrid")


## 1. 基本評価指標の手動実装


In [ ]:
def calculate_metrics_manual(y_true, y_pred):
    """
    基本評価指標の手動実装
    
    Parameters:
    y_true: 実際のラベル
    y_pred: 予測ラベル
    
    Returns:
    metrics: 評価指標の辞書
    """
    # 混同行列の計算
    cm = confusion_matrix(y_true, y_pred)
    
    if cm.shape == (2, 2):
        tn, fp, fn, tp = cm.ravel()
        
        # 基本指標の計算
        accuracy = (tp + tn) / (tp + tn + fp + fn)
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        
        return {
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall,
            'Specificity': specificity,
            'F1-score': f1,
            'TP': tp,
            'TN': tn,
            'FP': fp,
            'FN': fn
        }
    else:
        return None

# 簡単な例で指標を計算
y_true_example = np.array([1, 0, 1, 1, 0, 1, 0, 0, 1, 0])
y_pred_example = np.array([1, 0, 0, 1, 0, 1, 1, 0, 1, 0])

metrics_manual = calculate_metrics_manual(y_true_example, y_pred_example)

print("=== 基本評価指標の手動実装 ===")
print(f"実際のラベル: {y_true_example}")
print(f"予測ラベル: {y_pred_example}")
print(f"\n混同行列:")
cm_example = confusion_matrix(y_true_example, y_pred_example)
print(cm_example)

if metrics_manual:
    print(f"\n計算された指標:")
    for metric, value in metrics_manual.items():
        if isinstance(value, float):
            print(f"{metric}: {value:.4f}")
        else:
            print(f"{metric}: {value}")

# scikit-learnとの比較
accuracy_sklearn = accuracy_score(y_true_example, y_pred_example)
precision_sklearn = precision_score(y_true_example, y_pred_example)
recall_sklearn = recall_score(y_true_example, y_pred_example)
f1_sklearn = f1_score(y_true_example, y_pred_example)

print(f"\n=== scikit-learnとの比較 ===")
print(f"Accuracy: 手動={metrics_manual['Accuracy']:.4f}, sklearn={accuracy_sklearn:.4f}")
print(f"Precision: 手動={metrics_manual['Precision']:.4f}, sklearn={precision_sklearn:.4f}")
print(f"Recall: 手動={metrics_manual['Recall']:.4f}, sklearn={recall_sklearn:.4f}")
print(f"F1-score: 手動={metrics_manual['F1-score']:.4f}, sklearn={f1_sklearn:.4f}")


## 2. 指標の可視化と解釈


In [ ]:
def plot_metrics_comparison(metrics_dict, title="Evaluation Metrics"):
    """
    評価指標の可視化
    
    Parameters:
    metrics_dict: 評価指標の辞書
    title: グラフのタイトル
    """
    # 数値指標のみを抽出
    numeric_metrics = {k: v for k, v in metrics_dict.items() if isinstance(v, float)}
    
    # 可視化
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # バープロット
    metrics_names = list(numeric_metrics.keys())
    metrics_values = list(numeric_metrics.values())
    
    bars = ax1.bar(metrics_names, metrics_values, color=['skyblue', 'lightcoral', 'lightgreen', 'gold', 'plum'])
    ax1.set_title(f'{title} - Bar Plot')
    ax1.set_ylabel('Score')
    ax1.set_ylim(0, 1)
    ax1.grid(True, alpha=0.3)
    
    # 数値の表示
    for bar, value in zip(bars, metrics_values):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                f'{value:.3f}', ha='center', va='bottom')
    
    # レーダーチャート
    angles = np.linspace(0, 2 * np.pi, len(metrics_names), endpoint=False).tolist()
    angles += angles[:1]  # 閉じるため
    
    values = list(metrics_values) + [metrics_values[0]]  # 閉じるため
    
    ax2 = plt.subplot(122, projection='polar')
    ax2.plot(angles, values, 'o-', linewidth=2, color='blue')
    ax2.fill(angles, values, alpha=0.25, color='blue')
    ax2.set_xticks(angles[:-1])
    ax2.set_xticklabels(metrics_names)
    ax2.set_ylim(0, 1)
    ax2.set_title(f'{title} - Radar Chart')
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()

# 指標の可視化
if metrics_manual:
    plot_metrics_comparison(metrics_manual, "Basic Evaluation Metrics")

# 指標の解釈
print("=== 指標の解釈 ===")
print("Accuracy (精度): 全予測のうち正解した割合")
print("Precision (適合率): 正例と予測したもののうち、実際に正例だった割合")
print("Recall (再現率): 実際の正例のうち、正しく正例と予測できた割合")
print("Specificity (特異度): 実際の負例のうち、正しく負例と予測できた割合")
print("F1-score: PrecisionとRecallの調和平均")

# 実務での意味
print("\n=== 実務での意味 ===")
print("医療診断の例:")
print("- Precision: 病気と診断した患者のうち、実際に病気だった割合")
print("- Recall: 病気の患者のうち、正しく病気と診断できた割合")
print("- Specificity: 健康な患者のうち、正しく健康と診断できた割合")

print("\nスパム判定の例:")
print("- Precision: スパムと判定したメールのうち、実際にスパムだった割合")
print("- Recall: スパムメールのうち、正しくスパムと判定できた割合")


## 3. 実データでの評価指標


In [ ]:
# 乳がんデータセットでの評価
cancer = load_breast_cancer()
X, y = cancer.data, cancer.target

# データの分割と標準化
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ロジスティック回帰モデルの訓練
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(X_train_scaled, y_train)

# 予測
y_pred = model.predict(X_test_scaled)
y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]

# 評価指標の計算
metrics_cancer = calculate_metrics_manual(y_test, y_pred)

print("=== 乳がんデータセットでの評価 ===")
print(f"データセットの情報:")
print(f"  サンプル数: {len(y_test)}")
print(f"  クラス分布: {np.bincount(y_test)}")
print(f"  クラス名: {cancer.target_names}")

if metrics_cancer:
    print(f"\n評価指標:")
    for metric, value in metrics_cancer.items():
        if isinstance(value, float):
            print(f"  {metric}: {value:.4f}")
        else:
            print(f"  {metric}: {value}")

# 混同行列の可視化
cm_cancer = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_cancer, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Malignant', 'Benign'],
            yticklabels=['Malignant', 'Benign'])
plt.title('Confusion Matrix - Breast Cancer Dataset')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

# 分類レポート
print("\n=== 詳細な分類レポート ===")
print(classification_report(y_test, y_pred, target_names=cancer.target_names))

# 指標の可視化
if metrics_cancer:
    plot_metrics_comparison(metrics_cancer, "Breast Cancer Classification Metrics")


## 4. 閾値の影響


In [ ]:
def evaluate_thresholds(y_true, y_pred_proba, thresholds):
    """
    異なる閾値での評価指標を計算
    
    Parameters:
    y_true: 実際のラベル
    y_pred_proba: 予測確率
    thresholds: 閾値のリスト
    
    Returns:
    results: 各閾値での評価指標
    """
    results = []
    
    for threshold in thresholds:
        y_pred_thresh = (y_pred_proba >= threshold).astype(int)
        metrics = calculate_metrics_manual(y_true, y_pred_thresh)
        
        if metrics:
            metrics['Threshold'] = threshold
            results.append(metrics)
    
    return results

# 異なる閾値での評価
thresholds = np.arange(0.1, 1.0, 0.1)
threshold_results = evaluate_thresholds(y_test, y_pred_proba, thresholds)

# 結果の可視化
if threshold_results:
    # データフレームの作成
    df_thresholds = pd.DataFrame(threshold_results)
    
    # 指標の可視化
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Precision vs Threshold
    axes[0, 0].plot(df_thresholds['Threshold'], df_thresholds['Precision'], 'o-', color='blue')
    axes[0, 0].set_title('Precision vs Threshold')
    axes[0, 0].set_xlabel('Threshold')
    axes[0, 0].set_ylabel('Precision')
    axes[0, 0].grid(True, alpha=0.3)
    
    # Recall vs Threshold
    axes[0, 1].plot(df_thresholds['Threshold'], df_thresholds['Recall'], 'o-', color='red')
    axes[0, 1].set_title('Recall vs Threshold')
    axes[0, 1].set_xlabel('Threshold')
    axes[0, 1].set_ylabel('Recall')
    axes[0, 1].grid(True, alpha=0.3)
    
    # F1-score vs Threshold
    axes[1, 0].plot(df_thresholds['Threshold'], df_thresholds['F1-score'], 'o-', color='green')
    axes[1, 0].set_title('F1-score vs Threshold')
    axes[1, 0].set_xlabel('Threshold')
    axes[1, 0].set_ylabel('F1-score')
    axes[1, 0].grid(True, alpha=0.3)
    
    # Precision vs Recall
    axes[1, 1].plot(df_thresholds['Recall'], df_thresholds['Precision'], 'o-', color='purple')
    axes[1, 1].set_title('Precision vs Recall')
    axes[1, 1].set_xlabel('Recall')
    axes[1, 1].set_ylabel('Precision')
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # 最適な閾値の選択
    best_f1_idx = df_thresholds['F1-score'].idxmax()
    best_threshold = df_thresholds.loc[best_f1_idx, 'Threshold']
    best_f1 = df_thresholds.loc[best_f1_idx, 'F1-score']
    
    print(f"=== 閾値の影響分析 ===")
    print(f"最適なF1-score: {best_f1:.4f} (閾値: {best_threshold:.1f})")
    print(f"\n各閾値での指標:")
    print(df_thresholds[['Threshold', 'Precision', 'Recall', 'F1-score']].round(4))
    
    # 閾値の解釈
    print(f"\n=== 閾値の解釈 ===")
    print("低い閾値 (0.1-0.3):")
    print("  - Recall ↑, Precision ↓")
    print("  - より多くの正例を検出")
    print("  - 偽陽性が増加")
    
    print("\n高い閾値 (0.7-0.9):")
    print("  - Precision ↑, Recall ↓")
    print("  - より確実な正例のみ検出")
    print("  - 偽陰性が増加")
    
    print(f"\n最適な閾値 ({best_threshold:.1f}):")
    print("  - PrecisionとRecallのバランス")
    print("  - F1-scoreが最大")


## 5. 演習問題

### 演習1: 指標の手動実装
異なるデータセットで基本評価指標を手動実装し、scikit-learnの結果と比較してみましょう。

### 演習2: 閾値の最適化
医療診断ではRecallを重視し、スパム判定ではPrecisionを重視する場合の最適な閾値を探してみましょう。

### 演習3: クラス不均衡の影響
クラス不均衡データで基本評価指標を計算し、指標の変化を観察してみましょう。


## まとめ

このノートブックでは、分類問題の基本評価指標を詳しく学習しました。

**学習した内容**：
- 基本評価指標の理論と実装
- 手動実装とscikit-learnの比較
- 実データでの指標計算と解釈
- 閾値の影響と最適化

**重要なポイント**：
- Accuracy、Precision、Recall、Specificity、F1-scoreの理解
- 各指標の実務での意味
- 閾値の調整による指標の変化
- 可視化による直感的な理解

**次のステップ**：
- Precision-Recall曲線の学習
- ROC曲線とAUCの理解
- 多クラス分類の評価指標
